In [40]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [41]:
df_geo = pd.read_csv('geography.csv')

df_vac = pd.read_csv('vaccinations.csv')

df_ep = pd.read_csv('epidemiology.csv')

df_index = pd.read_csv('index.csv')

In [42]:
df_geo.head()

,location_key,openstreetmap_id,latitude,longitude,elevation_m,area_sq_km,area_rural_sq_km,area_urban_sq_km
0,AD,9407.0,42.558333,1.555278,NaN,470.0,NaN,NaN
1,AE,307763.0,24.400000,54.300000,NaN,83600.0,70575.0,8568.0
2,AF,303427.0,33.000000,66.000000,NaN,652860.0,NaN,NaN
3,AF_BAL,1674795.0,36.700000,67.116667,340.0,16186.0,NaN,NaN
4,AF_BAM,NaN,34.750000,67.250000,3042.0,14175.0,NaN,NaN


In [43]:
df_geo = df_geo[['location_key', 'latitude', 'longitude']]

In [44]:
df_vac.head()

,date,location_key,new_persons_vaccinated,cumulative_persons_vaccinated,new_persons_fully_vaccinated,cumulative_persons_fully_vaccinated,new_vaccine_doses_administered,cumulative_vaccine_doses_administered,new_persons_vaccinated_pfizer,cumulative_persons_vaccinated_pfizer,...,new_persons_fully_vaccinated_janssen,cumulative_persons_fully_vaccinated_janssen,new_vaccine_doses_administered_janssen,cumulative_vaccine_doses_administered_janssen,new_persons_vaccinated_sinovac,total_persons_vaccinated_sinovac,new_persons_fully_vaccinated_sinovac,total_persons_fully_vaccinated_sinovac,new_vaccine_doses_administered_sinovac,total_vaccine_doses_administered_sinovac
0,2021-01-25,AD,NaN,576.0,NaN,NaN,NaN,576.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-02-01,AD,460.0,1036.0,NaN,NaN,460.0,1036.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-02-10,AD,255.0,1291.0,NaN,NaN,255.0,1291.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-02-12,AD,331.0,1622.0,NaN,NaN,331.0,1622.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-02-19,AD,519.0,2141.0,NaN,NaN,519.0,2141.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df_vac = df_vac[['location_key', 'date', 'new_persons_fully_vaccinated']]

In [46]:
df_ep.head()

,date,location_key,new_confirmed,new_deceased,new_recovered,new_tested,cumulative_confirmed,cumulative_deceased,cumulative_recovered,cumulative_tested
0,2020-01-01,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
1,2020-01-02,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
2,2020-01-03,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
3,2020-01-04,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
4,2020-01-05,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN


In [47]:
df_ep = df_ep[['location_key', 'date', 'new_confirmed', 'new_deceased', 'new_recovered']]

In [51]:
df_index.columns

Index(['location_key', 'place_id', 'wikidata_id', 'datacommons_id',
       'country_code', 'country_name', 'subregion1_code', 'subregion1_name',
       'subregion2_code', 'subregion2_name', 'locality_code', 'locality_name',
       'iso_3166_1_alpha_2', 'iso_3166_1_alpha_3', 'aggregation_level'],
      dtype='object')

In [53]:
df_index = df_index[
    ["location_key", "country_name", "subregion1_name", "iso_3166_1_alpha_3"]
]

In [54]:
df_combined = pd.merge(df_geo, df_index, on='location_key')

df_combined = pd.merge(df_combined, df_vac, on=['location_key'], how='left')

df_combined = pd.merge(df_combined, df_ep, on=['location_key', 'date'], how='left')

In [55]:
geojson = gpd.read_file("custom.geo.json")
geometry = [Point(xy) for xy in zip(df_combined.longitude, df_combined.latitude)]
geo_df = gpd.GeoDataFrame(df_combined, geometry=geometry)

joined_data = gpd.sjoin(geo_df, geojson, how="left", op="within") 

DriverError: custom.geo.json: No such file or directory

In [56]:
df_combined.to_csv('processed.csv', index=False)

In [62]:
df_combined = df_combined[~df_combined.date.isna()]

In [66]:
df_combined[df_combined.iso_3166_1_alpha_3.str.contains("PAK", na=False)]

,location_key,latitude,longitude,country_name,subregion1_name,iso_3166_1_alpha_3,date,new_persons_fully_vaccinated,new_confirmed,new_deceased,new_recovered
710167,PK,30.0,71.0,Pakistan,NaN,PAK,2021-02-02,NaN,1384.0,56.0,NaN
710168,PK,30.0,71.0,Pakistan,NaN,PAK,2021-02-10,NaN,1502.0,57.0,NaN
710169,PK,30.0,71.0,Pakistan,NaN,PAK,2021-02-17,NaN,1272.0,52.0,NaN
710170,PK,30.0,71.0,Pakistan,NaN,PAK,2021-02-21,NaN,1160.0,16.0,NaN
710171,PK,30.0,71.0,Pakistan,NaN,PAK,2021-03-14,NaN,2253.0,29.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
710462,PK,30.0,71.0,Pakistan,NaN,PAK,2022-09-09,21144.0,156.0,1.0,NaN
710463,PK,30.0,71.0,Pakistan,NaN,PAK,2022-09-10,20065.0,139.0,0.0,NaN
710464,PK,30.0,71.0,Pakistan,NaN,PAK,2022-09-12,25041.0,99.0,1.0,NaN
710465,PK,30.0,71.0,Pakistan,NaN,PAK,2022-09-13,20745.0,235.0,4.0,NaN


In [85]:
df_test = df_combined[['iso_3166_1_alpha_3', 'date', 'new_confirmed']]

In [79]:
df_test = df_test.groupby(['iso_3166_1_alpha_3']).sum().reset_index()

In [84]:
df_test['date'] = pd.to_datetime(df_test['date'])

/var/folders/rj/gw4fw5gd6kn8_l_25b48hvhh0000gn/T/ipykernel_48835/1475905308.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['date'] = pd.to_datetime(df_test['date'])


In [70]:
# normalize data
df_test['new_confirmed'] = df_test['new_confirmed'] / df_test['new_confirmed'].max()

/var/folders/rj/gw4fw5gd6kn8_l_25b48hvhh0000gn/T/ipykernel_48835/3763663562.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['new_confirmed'] = df_test['new_confirmed'] / df_test['new_confirmed'].max()


In [86]:
df_test.to_csv('test.csv')